In [1]:
# This script converts the trained "model.h5" into OpenVINO MO.
# https://docs.openvino.ai/2022.3/openvino_docs_MO_DG_prepare_model_convert_model_Convert_Model_From_TensorFlow.html#keras-h5 
from pathlib import Path

In [2]:
model_h5_dir = Path("./files/A_20240907_200Epoch")
model_h5_path = model_h5_dir / "model-200epochs.h5"

In [3]:
import tensorflow as tf
from metrics import iou, dice_coef, dice_loss, bce_dice_loss
from tensorflow.keras.utils import CustomObjectScope

# Load the model
with CustomObjectScope({
        'iou':iou,
        'dice_coef':dice_coef,
        'dice_loss':dice_loss,
        'bce_dice_loss': bce_dice_loss
    }):
    try:
        h5_model = tf.keras.models.load_model(model_h5_path)
        print("Model loaded successfully.")
    except Exception as e:
        print(f"Failed to load model: {e}")
        

Model loaded successfully.


In [4]:
saved_model_path =  model_h5_dir /'model'
tf.saved_model.save(h5_model,saved_model_path)


INFO:tensorflow:Assets written to: files\A_20240907_200Epoch\model\assets


INFO:tensorflow:Assets written to: files\A_20240907_200Epoch\model\assets


In [5]:
import openvino as ov
ov_model = ov.convert_model(saved_model_path)

In [6]:
fn = str(saved_model_path /'nanonet-fp32-shape-1-256-256-3-model.xml')
ov.save_model(ov_model, fn)

In [7]:
# Open the file and read the first 20 lines
with open(fn, 'r') as file:
    for i in range(50):
        line = file.readline()
        print(line.strip())  # Using strip() to remove any extra newline characters

<?xml version="1.0"?>
<net name="TensorFlow_Frontend_IR" version="11">
<layers>
<layer id="0" name="input_image" type="Parameter" version="opset1">
<data shape="?,256,256,3" element_type="f32" />
<output>
<port id="0" precision="FP32" names="input_image">
<dim>-1</dim>
<dim>256</dim>
<dim>256</dim>
<dim>3</dim>
</port>
</output>
</layer>
<layer id="1" name="Constant_654" type="Const" version="opset1">
<data element_type="i64" shape="4" offset="0" size="32" />
<output>
<port id="0" precision="I64">
<dim>4</dim>
</port>
</output>
</layer>
<layer id="2" name="Transpose_655" type="Transpose" version="opset1">
<input>
<port id="0" precision="FP32">
<dim>-1</dim>
<dim>256</dim>
<dim>256</dim>
<dim>3</dim>
</port>
<port id="1" precision="I64">
<dim>4</dim>
</port>
</input>
<output>
<port id="2" precision="FP32">
<dim>-1</dim>
<dim>3</dim>
<dim>256</dim>
<dim>256</dim>
</port>
</output>
</layer>
<layer id="3" name="Multiply_17849_compressed" type="Const" version="opset1">
<data element_type="f

In [8]:
import openvino as ov
ov_model = ov.convert_model(saved_model_path)

In [9]:
# Convert Input Layout to N,C,H,W
prep = ov.preprocess.PrePostProcessor(ov_model)
input_name = ov_model.input().get_any_name()
print("input_name is", input_name)
prep.input(input_name).tensor().set_layout(ov.Layout("nchw"))
prep.input(input_name).model().set_layout(ov.Layout("nhwc"))
ov_model = prep.build()


input_name is input_image


In [10]:
fn = str(saved_model_path /'nanonet-fp32-shape-1-3-256-256-model.xml')
ov.save_model(ov_model, fn)

In [11]:
# Open the file and read the first 20 lines
with open(fn, 'r') as file:
    for i in range(50):
        line = file.readline()
        print(line.strip())  # Using strip() to remove any extra newline characters

<?xml version="1.0"?>
<net name="TensorFlow_Frontend_IR" version="11">
<layers>
<layer id="0" name="input_image" type="Parameter" version="opset1">
<data shape="?,3,256,256" element_type="f32" />
<output>
<port id="0" precision="FP32" names="input_image">
<dim>-1</dim>
<dim>3</dim>
<dim>256</dim>
<dim>256</dim>
<rt_info>
<attribute name="layout" version="0" layout="[N,C,H,W]" />
</rt_info>
</port>
</output>
</layer>
<layer id="1" name="Constant_37209" type="Const" version="opset1">
<data element_type="i64" shape="4" offset="0" size="32" />
<output>
<port id="0" precision="I64">
<dim>4</dim>
</port>
</output>
</layer>
<layer id="2" name="Transpose_37210" type="Transpose" version="opset1">
<input>
<port id="0" precision="FP32">
<dim>-1</dim>
<dim>3</dim>
<dim>256</dim>
<dim>256</dim>
</port>
<port id="1" precision="I64">
<dim>4</dim>
</port>
</input>
<output>
<port id="2" precision="FP32">
<dim>-1</dim>
<dim>256</dim>
<dim>256</dim>
<dim>3</dim>
</port>
</output>
</layer>
<layer id="3" n